In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import sys
import random
import warnings
import zipfile
import cv2
import glob

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, BatchNormalization, ELU
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate, Add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Set some parameters
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
#seed = 1.
#random.seed = seed
#np.random.seed = seed
smooth = 1.
epochs = 50
%matplotlib inline

Found GPU at: /device:GPU:0


In [0]:
#with zipfile.ZipFile('gdrive/My Drive/Colab Notebooks/coursachCV/images1.zip', 'r') as zip_images:
#    zip_images.extractall()
#with zipfile.ZipFile('gdrive/My Drive/Colab Notebooks/coursachCV/images2.zip', 'r') as zip_images:
#    zip_images.extractall()
#with zipfile.ZipFile('gdrive/My Drive/Colab Notebooks/coursachCV/labels1.zip', 'r') as zip_labels:
#    zip_labels.extractall()
#with zipfile.ZipFile('gdrive/My Drive/Colab Notebooks/coursachCV/images_ext.zip', 'r') as zip_images:
#    zip_images.extractall()
#with zipfile.ZipFile('gdrive/My Drive/Colab Notebooks/coursachCV/labels_ext.zip', 'r') as zip_labels:
#    zip_labels.extractall()
with zipfile.ZipFile('gdrive/My Drive/Colab Notebooks/coursachCV/images_full.zip', 'r') as zip_images:
    zip_images.extractall()
with zipfile.ZipFile('gdrive/My Drive/Colab Notebooks/coursachCV/labels_full.zip', 'r') as zip_labels:
    zip_labels.extractall()

In [0]:
id2code = {0: (0, 0, 0),
           1: (111, 74, 0),
           2: (81, 0, 81),
           3: (128, 64, 128),
           4: (244, 35, 232),
           5: (250, 170, 160),
           6: (230, 150, 140),
           7: (70, 70, 70),
           8: (102, 102, 156),
           9: (190, 153, 153),
           10: (180, 165, 180),
           11: (150, 100, 100),
           12: (150, 120, 90),
           13: (153, 153, 153),
           14: (250, 170, 30),
           15: (220, 220, 0),
           16: (107, 142, 35),
           17: (152, 251, 152),
           18: (70, 130, 180),
           19: (220, 20, 60),
           20: (255, 0, 0),
           21: (0, 0, 142),
           22: (0, 0, 70),
           23: (0, 60, 100),
           24: (0, 0, 90),
           25: (0, 0, 110),
           26: (0, 80, 100),
           27: (0, 0, 230),
           28: (119, 11, 32)}

"""
id2code = {0: [(0, 0, 0),(111, 74, 0),(81, 0, 81)],
           1: [(128, 64, 128),(244, 35, 232),(250, 170, 160),(230, 150, 140)],
           2: [(70, 70, 70),(102, 102, 156),(190, 153, 153),(180, 165, 180),(150, 100, 100),(150, 120, 90)],
           3: [(153, 153, 153),(250, 170, 30),(220, 220, 0)],
           4: [(107, 142, 35),(152, 251, 152)],
           5: [(70, 130, 180)],
           6: [(220, 20, 60),(255, 0, 0)],
           7: [(0, 0, 142),(0, 0, 70),(0, 60, 100),(0, 0, 90),(0, 0, 110),(0, 80, 100),(0, 0, 230),(119, 11, 32)]}
"""
print(id2code[0][0])

def rgb_to_onehot(rgb_image, colormap=id2code):
    num_classes = len(colormap)
    shape = rgb_image.shape[:2]+(num_classes,)
    encoded_image = np.zeros(shape, dtype=np.int8)
    for i, cls in enumerate(colormap):
        encoded_image[:,:,i] = np.all(rgb_image.reshape( (-1,3) ) == colormap[i], axis=1).reshape(shape[:2])
    return encoded_image

def onehot_to_rgb(onehot, colormap = id2code):
    single_layer = np.argmax(onehot, axis=-1)
    output = np.zeros( onehot.shape[:2]+(3,) )
    for k in colormap.keys():
        output[single_layer==k] = colormap[k]
    return np.uint8(output)

0


In [0]:
data_gen_args = dict(rescale=1./255)
mask_gen_args = dict()
train_frames_datagen = ImageDataGenerator(**data_gen_args)
train_masks_datagen = ImageDataGenerator(**mask_gen_args)
val_frames_datagen = ImageDataGenerator(**data_gen_args)
val_masks_datagen = ImageDataGenerator(**mask_gen_args)

def TrainAugmentGenerator(seed = 1, batch_size = 32):
    train_image_generator = train_frames_datagen.flow_from_directory(
    directory='train_frames/', batch_size = batch_size, seed = seed)
    train_mask_generator = train_masks_datagen.flow_from_directory(
    directory='train_masks/', batch_size = batch_size, seed = seed)
    while True:
        X1i = train_image_generator.next()
        X2i = train_mask_generator.next()
        mask_encoded = [rgb_to_onehot(X2i[0][x,:,:,:], id2code) for x in range(X2i[0].shape[0])]
        yield X1i[0], np.asarray(mask_encoded)

def ValAugmentGenerator(seed = 1, batch_size = 32):
    val_image_generator = val_frames_datagen.flow_from_directory(directory='val_frames/',
    batch_size = batch_size, seed = seed)
    val_mask_generator = val_masks_datagen.flow_from_directory(directory='val_masks/',
    batch_size = batch_size, seed = seed)
    while True:
        X1i = val_image_generator.next()
        X2i = val_mask_generator.next()
        mask_encoded = [rgb_to_onehot(X2i[0][x,:,:,:], id2code) for x in range(X2i[0].shape[0])] 
        yield X1i[0], np.asarray(mask_encoded)

In [0]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f*y_true_f) + K.sum(y_pred_f*y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1.-dice_coef(y_true, y_pred)

In [0]:
def get_unet():
    inputs = Input(shape=(256, 256, 3))

    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (inputs)
    c1 = BatchNormalization() (c1)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (c1)
    c1 = BatchNormalization() (c1)
    p1 = MaxPooling2D((2, 2), data_format='channels_last') (c1)

    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (p1)
    c2 = BatchNormalization() (c2)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (c2)
    c2 = BatchNormalization() (c2)
    p2 = MaxPooling2D((2, 2), data_format='channels_last') (c2)

    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (p2)
    c3 = BatchNormalization() (c3)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (c3)
    c3 = BatchNormalization() (c3)
    p3 = MaxPooling2D((2, 2), data_format='channels_last') (c3)

    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (p3)
    c4 = BatchNormalization() (c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (c4)
    c4 = BatchNormalization() (c4)
    p4 = MaxPooling2D(pool_size=(2, 2), data_format='channels_last') (c4)

    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (p4)
    c5 = BatchNormalization() (c5)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (c5)
    c5 = BatchNormalization() (c5)

    # u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = UpSampling2D(size=(2, 2)) (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (u6)
    c6 = BatchNormalization() (c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (c6)
    c6 = BatchNormalization() (c6)

    #u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = UpSampling2D(size=(2, 2)) (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (u7)
    c7 = BatchNormalization() (c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (c7)
    c7 = BatchNormalization() (c7)

    #u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = UpSampling2D(size=(2, 2)) (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (u8)
    c8 = BatchNormalization() (c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (c8)
    c8 = BatchNormalization() (c8)

    #u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = UpSampling2D(size=(2, 2)) (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (u9)
    c9 = BatchNormalization() (c9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same', dilation_rate = 1) (c9)
    c9 = BatchNormalization() (c9)

    outputs = Conv2D(29, (1, 1), activation='softmax', padding = 'same', dilation_rate = 1) (c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[dice_coef])
    return model

In [0]:
u_net = get_unet()

In [0]:
dependencies = {
     'dice_coef': dice_coef
}
#u_net = load_model('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_1.h5', custom_objects=dependencies)
u_net.summary()
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_1.h5', verbose=1, save_best_only=True)
results = u_net.fit_generator(TrainAugmentGenerator(batch_size=32), validation_data = ValAugmentGenerator(batch_size=32),
                              steps_per_epoch=2000//32, validation_steps=500//32, epochs=50, callbacks=[earlystopper, checkpointer])

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 256, 256, 16) 448         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_54 (BatchNo (None, 256, 256, 16) 64          conv2d_57[0][0]                  
__________________________________________________________________________________________________
conv2d_58 (Conv2D)              (None, 256, 256, 16) 2320        batch_normalization_54[0][0]     
____________________________________________________________________________________________

In [0]:
dependencies = {
     'dice_coef': dice_coef,
     'dice_coef_loss': dice_coef_loss
}
u_net = load_model('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_1.h5', custom_objects=dependencies)

In [0]:
for i in range(0,np.shape(pred_all)[0]):
    
    fig = plt.figure(figsize=(20,8))
    
    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(batch_img[i])
    ax1.title.set_text('Actual frame')
    ax1.grid(b=None)
    
    
    ax2 = fig.add_subplot(1,3,2)
    ax2.set_title('Ground truth labels')
    ax2.imshow(onehot_to_rgb(batch_mask[i],id2code))
    ax2.grid(b=None)
    
    ax3 = fig.add_subplot(1,3,3)
    ax3.set_title('Predicted labels')
    ax3.imshow(onehot_to_rgb(pred_all[i],id2code))
    ax3.grid(b=None)
    
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [0]:
dependencies = {
     'dice_coef': dice_coef,
     'dice_coef_loss': dice_coef_loss
}
u_net = load_model('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_1.h5', custom_objects=dependencies)
u_net.summary()
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_2.h5', verbose=1, save_best_only=True)
results = u_net.fit_generator(TrainAugmentGenerator(batch_size=32), validation_data = ValAugmentGenerator(batch_size=32),
                              steps_per_epoch=4000//32, validation_steps=1000//32, epochs=50, callbacks=[earlystopper, checkpointer])

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 256, 256, 16) 448         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_54 (BatchNo (None, 256, 256, 16) 64          conv2d_57[0][0]                  
__________________________________________________________________________________________________
conv2d_58 (Conv2D)              (None, 256, 256, 16) 2320        batch_normalization_54[0][0]     
____________________________________________________________________________________________

In [0]:
dependencies = {
     'dice_coef': dice_coef
}
u_net = load_model('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_2.h5', custom_objects=dependencies)
testing_gen = ValAugmentGenerator()
batch_img, batch_mask = next(testing_gen)
pred_all = u_net.predict(batch_img)
np.shape(pred_all)
for i in range(0,np.shape(pred_all)[0]):
    
    fig = plt.figure(figsize=(20,8))
    
    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(batch_img[i])
    ax1.title.set_text('Actual frame')
    ax1.grid(b=None)
    
    
    ax2 = fig.add_subplot(1,3,2)
    ax2.set_title('Ground truth labels')
    ax2.imshow(onehot_to_rgb(batch_mask[i],id2code))
    ax2.grid(b=None)
    
    ax3 = fig.add_subplot(1,3,3)
    ax3.set_title('Predicted labels')
    ax3.imshow(onehot_to_rgb(pred_all[i],id2code))
    ax3.grid(b=None)
    
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [0]:
def inc(layer_in, n_filters = 16):
	# 1x1 conv
    conv1 = BatchNormalization()(layer_in)
    conv1 = Conv2D(n_filters, (1,1), padding='same', activation='elu')(conv1)
	# 3x3 conv
    conv3 = Conv2D(n_filters, (1,1), padding='same', activation='elu')(layer_in)
    conv3 = Conv2D(n_filters, (3,3), padding='same', activation='elu')(conv3)
	# 5x5 conv
    conv5 = Conv2D(n_filters, (1,1), padding='same', activation='elu')(layer_in)
    conv5 = Conv2D(n_filters, (5,5), padding='same', activation='elu')(conv5)
	# 3x3 max pooling
    pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
    pool = BatchNormalization()(pool)
    pool = Conv2D(n_filters, (1,1), padding='same', activation='elu')(pool)
	# concatenate filters, assumes filters/channels last
    layer_out = concatenate([conv1, conv3, conv5, pool], axis=3)
    return layer_out

def get_unet_inc():
    inputs = Input(shape=(256, 256, 3))

    c1 = inc(inputs, 16)
    p1 = MaxPooling2D((2,2))(c1)

    c2 = inc(p1, 32)
    p2 = MaxPooling2D((2,2))(c2)

    c3 = inc(p2, 64)
    p3 = MaxPooling2D((2,2))(c3)

    c4 = inc(p3, 128)
    p4 = MaxPooling2D((2,2))(c4)

    c5 = inc(p4, 256)

    # u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = UpSampling2D(size=(2,2))(c5)
    u6 = concatenate([u6, c4], axis=3)
    c6 = inc(u6, 128)

    #u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = UpSampling2D(size=(2,2))(c6)
    u7 = concatenate([u7, c3], axis=3)
    c7 = inc(u7, 64)

    #u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = UpSampling2D(size=(2,2))(c7)
    u8 = concatenate([u8, c2], axis=3)
    c8 = inc(u8, 32)

    #u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = UpSampling2D(size=(2,2))(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = inc(u9, 16)

    outputs = Conv2D(29, (1,1), activation='softmax', padding='same')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[dice_coef])
    return model

In [0]:
u_net_inc = get_unet_inc()
u_net_inc.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 3)  12          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 16) 64          input_1[0][0]                    
______________________________________________________________________________________________

In [0]:
dependencies = {
     'dice_coef': dice_coef,
     'dice_coef_loss': dice_coef_loss
}
#u_net_inc = load_model('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_inc_3.h5', custom_objects=dependencies)
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_inc_2.h5', verbose=1, save_best_only=True)
results = u_net_inc.fit(TrainAugmentGenerator(batch_size=8), validation_data = ValAugmentGenerator(batch_size=8),
                              steps_per_epoch=2000//8, validation_steps=500//8, epochs=50, callbacks=[earlystopper, checkpointer])

Found 2000 images belonging to 1 classes.
Found 2000 images belonging to 1 classes.
Epoch 1/50
250/250 [==============================] - ETA: 0s - loss: 0.8108 - dice_coef: 0.8148Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.

Epoch 00001: val_loss improved from inf to 2.21393, saving model to gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_inc_2.h5
250/250 [==============================] - 283s 1s/step - loss: 0.8108 - dice_coef: 0.8148 - val_loss: 2.2139 - val_dice_coef: 0.5500
Epoch 2/50
250/250 [==============================] - ETA: 0s - loss: 0.6037 - dice_coef: 0.8622
Epoch 00002: val_loss improved from 2.21393 to 0.81451, saving model to gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_inc_2.h5
250/250 [==============================] - 282s 1s/step - loss: 0.6037 - dice_coef: 0.8622 - val_loss: 0.8145 - val_dice_coef: 0.8256
Epoch 3/50
250/250 [==============================] - ETA: 0s - loss: 0.5254 - dice_coef: 0.8796
Epoch 00003

In [0]:
dependencies = {
     'dice_coef': dice_coef,
     'dice_coef_loss': dice_coef_loss
}
u_net_inc = load_model('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_inc_2.h5', custom_objects=dependencies)

testing_gen = ValAugmentGenerator()
batch_img, batch_mask = next(testing_gen)
pred_all = u_net_inc.predict(batch_img)
np.shape(pred_all)

for i in range(0,np.shape(pred_all)[0]):
    
    fig = plt.figure(figsize=(20,8))
    
    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(batch_img[i])
    ax1.title.set_text('Actual frame')
    ax1.grid(b=None)
    
    
    ax2 = fig.add_subplot(1,3,2)
    ax2.set_title('Ground truth labels')
    ax2.imshow(onehot_to_rgb(batch_mask[i],id2code))
    ax2.grid(b=None)
    
    ax3 = fig.add_subplot(1,3,3)
    ax3.set_title('Predicted labels')
    ax3.imshow(onehot_to_rgb(pred_all[i],id2code))
    ax3.grid(b=None)
    
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [0]:
#dependencies = {
#     'dice_coef': dice_coef,
#     'dice_coef_loss': dice_coef_loss
#}
#u_net_inc = load_model('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_inc_3.h5', custom_objects=dependencies)
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_inc_3.h5', verbose=1, save_best_only=True)
results = u_net_inc.fit(TrainAugmentGenerator(batch_size=16), validation_data = ValAugmentGenerator(batch_size=16),
                              steps_per_epoch=4000//16, validation_steps=1000//16, epochs=50, callbacks=[earlystopper, checkpointer])

Found 4000 images belonging to 1 classes.
Found 4000 images belonging to 1 classes.
Epoch 1/50
250/250 [==============================] - ETA: 0s - loss: 0.7524 - dice_coef: 0.8275Found 1000 images belonging to 1 classes.
Found 1000 images belonging to 1 classes.

Epoch 00001: val_loss improved from inf to 2.85776, saving model to gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_inc_3.h5
250/250 [==============================] - 292s 1s/step - loss: 0.7524 - dice_coef: 0.8275 - val_loss: 2.8578 - val_dice_coef: 0.4889
Epoch 2/50
250/250 [==============================] - ETA: 0s - loss: 0.5430 - dice_coef: 0.8754
Epoch 00002: val_loss improved from 2.85776 to 0.77660, saving model to gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_inc_3.h5
250/250 [==============================] - 291s 1s/step - loss: 0.5430 - dice_coef: 0.8754 - val_loss: 0.7766 - val_dice_coef: 0.8156
Epoch 3/50
250/250 [==============================] - ETA: 0s - loss: 0.4812 - dice_coef: 0.8892
Epoch 000

In [0]:
dependencies = {
     'dice_coef': dice_coef,
     'dice_coef_loss': dice_coef_loss
}
u_net_inc = load_model('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_inc_3.h5', custom_objects=dependencies)

testing_gen = ValAugmentGenerator(batch_sizе = 16)
batch_img, batch_mask = next(testing_gen)
pred_all = u_net_inc.predict(batch_img)
np.shape(pred_all)

for i in range(0,np.shape(pred_all)[0]):
    
    fig = plt.figure(figsize=(20,8))
    
    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(batch_img[i])
    ax1.title.set_text('Actual frame')
    ax1.grid(b=None)
    
    
    ax2 = fig.add_subplot(1,3,2)
    ax2.set_title('Ground truth labels')
    ax2.imshow(onehot_to_rgb(batch_mask[i],id2code))
    ax2.grid(b=None)
    
    ax3 = fig.add_subplot(1,3,3)
    ax3.set_title('Predicted labels')
    ax3.imshow(onehot_to_rgb(pred_all[i],id2code))
    ax3.grid(b=None)
    
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [0]:
def conv_res(layer_in, n_filters = 16):
    conv1 = Conv2D(n_filters, (1, 1), activation='elu', padding='same')(layer_in)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(n_filters, (3, 3), activation='elu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(n_filters, (1, 1), padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv2 = Conv2D(n_filters, (1, 1), padding='same')(layer_in)
    conv2 = BatchNormalization()(conv2)
    layer_out = Add()([conv1, conv2])
    layer_out = ELU()(layer_out)
    return layer_out

def identity(layer_in, n_filters = 16):
    conv1 = Conv2D(n_filters, (1, 1), activation='elu', padding='same')(layer_in)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(n_filters, (3, 3), activation='elu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(n_filters, (1, 1), padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    layer_out = Add()([conv1, layer_in])
    layer_out = ELU()(layer_out)
    return layer_out

def get_unet_res():
    inputs = Input(shape=(256, 256, 3))

    #c0 = Conv2D(16, (3, 3), activation='elu', padding='same')(inputs)

    c1 = conv_res(inputs, 16)
    c1 = identity(c1, 16)
    c1 = identity(c1, 16)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = conv_res(p1, 32)
    c2 = identity(c2, 32)
    c2 = identity(c2, 32)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = conv_res(p2, 64)
    c3 = identity(c3, 64)
    c3 = identity(c3, 64)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = conv_res(p3, 128)
    c4 = identity(c4, 128)
    c4 = identity(c4, 128)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = conv_res(p4, 256)
    c5 = identity(c5, 256)
    c5 = identity(c5, 256)

    # u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = UpSampling2D(size=(2, 2)) (c5)
    u6 = concatenate([u6, c4])
    c6 = conv_res(u6, 128)
    c6 = identity(c6, 128)
    c6 = identity(c6, 128)

    #u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = UpSampling2D(size=(2, 2)) (c6)
    u7 = concatenate([u7, c3])
    c7 = conv_res(u7, 64)
    c7 = identity(c7, 64)
    c7 = identity(c7, 64)

    #u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = UpSampling2D(size=(2, 2)) (c7)
    u8 = concatenate([u8, c2])
    c8 = conv_res(u8, 32)
    c8 = identity(c8, 32)
    c8 = identity(c8, 32)

    #u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = UpSampling2D(size=(2, 2)) (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = conv_res(u9, 16)
    c9 = identity(c9, 16)
    c9 = identity(c9, 16)

    outputs = Conv2D(29, (1, 1), activation='softmax', padding = 'same')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[dice_coef])
    return model

In [0]:
#u_net_res = get_unet_res()
dependencies = {
     'dice_coef': dice_coef
}
u_net_res = load_model('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_res_4.h5', custom_objects=dependencies)
u_net_res.summary()
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_res_4.h5', verbose=1, save_best_only=True)
results = u_net_res.fit_generator(TrainAugmentGenerator(), validation_data = ValAugmentGenerator(),
                              steps_per_epoch=2000//16, validation_steps=500//16, epochs=50, callbacks=[earlystopper, checkpointer])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 256, 256, 16) 64          input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_45 (BatchNo (None, 256, 256, 16) 64          conv2d_61[0][0]                  
__________________________________________________________________________________________________
conv2d_62 (Conv2D)              (None, 256, 256, 16) 2320        batch_normalization_45[0][0]     
______________________________________________________________________________________________

In [0]:
dependencies = {
     'dice_coef': dice_coef
}
u_net_res = load_model('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_res_4.h5', custom_objects=dependencies)

testing_gen = ValAugmentGenerator()
batch_img, batch_mask = next(testing_gen)
pred_all = u_net_res.predict(batch_img)
np.shape(pred_all)

for i in range(0,np.shape(pred_all)[0]):
    
    fig = plt.figure(figsize=(20,8))
    
    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(batch_img[i])
    ax1.title.set_text('Actual frame')
    ax1.grid(b=None)
    
    
    ax2 = fig.add_subplot(1,3,2)
    ax2.set_title('Ground truth labels')
    ax2.imshow(onehot_to_rgb(batch_mask[i],id2code))
    ax2.grid(b=None)
    
    ax3 = fig.add_subplot(1,3,3)
    ax3.set_title('Predicted labels')
    ax3.imshow(onehot_to_rgb(pred_all[i],id2code))
    ax3.grid(b=None)
    
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [0]:
dependencies = {
     'dice_coef': dice_coef
}
u_net_res = load_model('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_res_5.h5', custom_objects=dependencies)
u_net_res.summary()
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_res_5.h5', verbose=1, save_best_only=True)
results = u_net_res.fit_generator(TrainAugmentGenerator(), validation_data = ValAugmentGenerator(),
                              steps_per_epoch=4000//32, validation_steps=1000//32, epochs=50, callbacks=[earlystopper, checkpointer])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 256, 256, 16) 64          input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_45 (BatchNo (None, 256, 256, 16) 64          conv2d_61[0][0]                  
__________________________________________________________________________________________________
conv2d_62 (Conv2D)              (None, 256, 256, 16) 2320        batch_normalization_45[0][0]     
______________________________________________________________________________________________

In [0]:
dependencies = {
     'dice_coef': dice_coef
}
u_net_res = load_model('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_res_5.h5', custom_objects=dependencies)

testing_gen = ValAugmentGenerator()
batch_img, batch_mask = next(testing_gen)
pred_all = u_net_res.predict(batch_img)
np.shape(pred_all)

for i in range(0,np.shape(pred_all)[0]):
    
    fig = plt.figure(figsize=(20,8))
    
    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(batch_img[i])
    ax1.title.set_text('Actual frame')
    ax1.grid(b=None)
    
    
    ax2 = fig.add_subplot(1,3,2)
    ax2.set_title('Ground truth labels')
    ax2.imshow(onehot_to_rgb(batch_mask[i],id2code))
    ax2.grid(b=None)
    
    ax3 = fig.add_subplot(1,3,3)
    ax3.set_title('Predicted labels')
    ax3.imshow(onehot_to_rgb(pred_all[i],id2code))
    ax3.grid(b=None)
    
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [0]:
dependencies = {
     'dice_coef': dice_coef
}
u_net_res = load_model('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_res_5.h5', custom_objects=dependencies)
u_net_res.summary()
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_res_5.h5', verbose=1, save_best_only=True)
results = u_net_res.fit_generator(TrainAugmentGenerator(), validation_data = ValAugmentGenerator(),
                              steps_per_epoch=20000//32, validation_steps=5000//32, epochs=50, callbacks=[earlystopper, checkpointer])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 256, 256, 16) 64          input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_45 (BatchNo (None, 256, 256, 16) 64          conv2d_61[0][0]                  
__________________________________________________________________________________________________
conv2d_62 (Conv2D)              (None, 256, 256, 16) 2320        batch_normalization_45[0][0]     
______________________________________________________________________________________________

In [0]:
dependencies = {
     'dice_coef': dice_coef
}
u_net_res = load_model('gdrive/My Drive/Colab Notebooks/coursachCV/game_unet_res_5.h5', custom_objects=dependencies)

testing_gen = ValAugmentGenerator()
batch_img, batch_mask = next(testing_gen)
pred_all = u_net_res.predict(batch_img)
np.shape(pred_all)

for i in range(0,np.shape(pred_all)[0]):
    
    fig = plt.figure(figsize=(20,8))
    
    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(batch_img[i])
    ax1.title.set_text('Actual frame')
    ax1.grid(b=None)
    
    
    ax2 = fig.add_subplot(1,3,2)
    ax2.set_title('Ground truth labels')
    ax2.imshow(onehot_to_rgb(batch_mask[i],id2code))
    ax2.grid(b=None)
    
    ax3 = fig.add_subplot(1,3,3)
    ax3.set_title('Predicted labels')
    ax3.imshow(onehot_to_rgb(pred_all[i],id2code))
    ax3.grid(b=None)
    
    plt.show()

Output hidden; open in https://colab.research.google.com to view.